In [1]:
import openl3

Using TensorFlow backend.


In [2]:
import openmic.vggish

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
import librosa

In [4]:
from tqdm import tqdm_notebook as tqdm

In [7]:
import soundfile as sf

In [13]:
import numpy as np

In [14]:
import os

---
# OpenL3 model

In [9]:
model = openl3.models.load_embedding_model(input_repr="mel128", content_type="music", embedding_size=512)

tracking <tf.Variable 'melspectrogram_1/real_kernels:0' shape=(2048, 1, 1, 1025) dtype=float32> dft_real_kernels
tracking <tf.Variable 'melspectrogram_1/imag_kernels:0' shape=(2048, 1, 1, 1025) dtype=float32> dft_imag_kernels
tracking <tf.Variable 'melspectrogram_1/Variable:0' shape=(1025, 128) dtype=float32> freq2mel



In [32]:
irmas_files = librosa.util.find_files('/home/bmcfee/data/irmas/IRMAS-TrainingData/')

In [33]:
outpath = '/home/bmcfee/working/wimir2019/irmas-openl3'

In [34]:
!mkdir -p {outpath}

In [35]:
for fn in tqdm(irmas_files):
    openl3.process_file(fn, output_dir=outpath, model=model, verbose=False)

In [40]:
openmic_files = librosa.util.find_files('/home/bmcfee/data/openmic-2018/audio/')

In [41]:
outpath = '/home/bmcfee/working/wimir2019/openmic-openl3'

In [42]:
!mkdir -p {outpath}

In [43]:
for fn in tqdm(openmic_files):
    prefix = os.path.basename(fn)[:3]
    
    dirname = os.path.sep.join([outpath, prefix])
    try:
        os.mkdir(dirname)
    except OSError:
        pass
    
    openl3.process_file(fn, output_dir=dirname, model=model, verbose=False)

---
# VGGish model

In [44]:
import numpy as np
import json

In [45]:
files = librosa.util.find_files('/home/bmcfee/data/irmas/vggish/', ext='npz')

In [58]:
!mkdir -p /home/bmcfee/working/wimir2019/irmas-vggish

In [61]:
for fn in tqdm(files):
    bn = os.path.basename(fn)
    base = os.path.splitext(os.path.splitext(bn)[0])[0]
    
    X = np.load(fn)
    
    out = dict(features=X['features_z'].tolist(), time_points=(0.96 * np.arange(len(X['features_z']))).tolist())
    with open('/home/bmcfee/working/wimir2019/irmas-vggish/{}.json'.format(base), 'w') as fd:
        json.dump(out, fd)